In [109]:
# # import Google earth engine module
# import ee

# #Authenticate the Google earth engine with google account
# ee.Initialize()

# #import module

# import folium

# #Define year
# startyear = 2019;
# endyear = 2019;

# #Create the Date object
# startdate = ee.Date.fromYMD(startyear,1,1);
# enddate = ee.Date.fromYMD(endyear,12,1);

# # Define a method for displaying Earth Engine image tiles on a folium map.
# def add_ee_layer(self, ee_object, vis_params, name):
    
#     try:    
#         # display ee.Image()
#         if isinstance(ee_object, ee.image.Image):    
#             map_id_dict = ee.Image(ee_object).getMapId(vis_params)
#             folium.raster_layers.TileLayer(
#             tiles = map_id_dict['tile_fetcher'].url_format,
#             attr = 'Google Earth Engine',
#             name = name,
#             overlay = True,
#             control = True
#             ).add_to(self)
#     except:
#         print("Could not display {}".format(name))

# my_map = folium.Map([41, 23], zoom_start=6, tiles='openstreetmap')
# # Add EE drawing method to folium.
# folium.Map.add_ee_layer = add_ee_layer

# #Load the SRTM image.
# srtm = ee.Image('USGS/SRTMGL1_003');

# #Apply slope algorithm to an image.
# slope = ee.Terrain.slope(srtm);

# #Scale and Projection
# scale = srtm.projection().nominalScale();

# #styling
# visualization_params = {min: 0, max: 10000, 'palette': [
#     '3ae237', 'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13',
#     'ff6e08', 'ff500d', 'ff0000', 'de0101', 'c21301', '0602ff', '235cb1',
#     '307ef3', '269db1', '30c8e2', '32d3ef', '3be285', '3ff38f', '86e26f'
#   ],};


# # Add the data to the map object.
# my_map.add_ee_layer(srtm, visualization_params ,"DEM");
# # my_map.add_ee_layer(slope, visualization_params , "slope");

# # Display the map.
# display(my_map)

In [3]:
import os
import ee
import geemap
from ipyleaflet import Map, basemaps, WidgetControl, Marker, projections
import json
import geopandas as gpd
import ee
import folium
ee.Initialize()

In [9]:
area = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              22.795257568359375,
              40.70823051511181
            ],
            [
              22.835769653320312,
              40.70823051511181
            ],
            [
              22.835769653320312,
              40.733730386116875
            ],
            [
              22.795257568359375,
              40.733730386116875
            ],
            [
              22.795257568359375,
              40.70823051511181
            ]
          ]
        ]
      }
    }
  ]
}

coords_area = area['features'][0]['geometry']['coordinates']
aoi_area = ee.Geometry.Polygon(coords_area)

station = [5.5730222, -0.24447074]

In [162]:
m = geemap.Map(zoom_start=6)
m.setCenter(23, 41)

#Load the SRTM image.
srtm = ee.Image('USGS/SRTMGL1_003');

#Apply slope algorithm to an image.
slope = ee.Terrain.slope(srtm);

visualization_params = {min: 0, max: 10000, 'palette': [
    '3ae237', 'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13',
    'ff6e08', 'ff500d', 'ff0000', 'de0101', 'c21301', '0602ff', '235cb1',
    '307ef3', '269db1', '30c8e2', '32d3ef', '3be285', '3ff38f', '86e26f'
  ],};

m.addLayer(slope.select(0), visualization_params)

# marker = Marker(location=(40.9, 22.3), draggable=True)
# m.add_layer(marker);

m

Map(center=[41, 23], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [7]:
x = float(input('X: '))
y = float(input('Y: '))
xy = ee.Geometry.Point([x, y])
slope = slope.sample(xy, 30).first().get('slope').getInfo()

print('Slope (%):', slope)

X: 40
Y: 23
Slope (%): 1.0074855089187622


In [ ]:
taskToexport = ee.batch.Export.image.toDrive(
        image = srtm,             
        region = aoi_area.bounds().getInfo()['coordinates'],
        description = 'Area',
        )

taskToexport.start()

In [108]:
taskToexport.status()

{'state': 'COMPLETED',
 'description': 'test1',
 'creation_timestamp_ms': 1614293742044,
 'update_timestamp_ms': 1614293765790,
 'start_timestamp_ms': 1614293755930,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/'],
 'attempt': 1,
 'id': 'CVV75KZVWK6WAM2LU7RILAEH',
 'name': 'projects/earthengine-legacy/operations/CVV75KZVWK6WAM2LU7RILAEH'}

In [160]:
def lights(years):
    # set our initial map parameters for Washington, DC
    center_lat = 39
    center_lon = 23.5
    zoomlevel = 6.3

    # initialize our map
    map1 = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)
    map1.add_basemap('SATELLITE')
    visualization_params = {min: 0, max: 10000, 'palette': [
        'ffffbf', 'cc5500', 'cc5500'
      ],};
    # get 1996 composite, apply mask, and add as layer
    dmsp = ee.Image("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/{}".format(years)).select('stable_lights')
    map1.addLayer(dmsp.mask(dmsp), {}, "DMSP-OLS 1996")
    map1.addLayerControl()
    display(map1)
    
from ipywidgets import interact, interactive, fixed, interact_manual

slider = widgets.SelectionSlider(
    options=['F101992', 'F101993', 'F121994', 'F121995', 'F121996', 'F141997', 'F141998', 'F141999', 'F152000', 'F152001', 'F152002', 'F152003', 'F162004', 'F162005', 'F162006', 'F162007', 'F162008', 'F162009', 'F182010', 'F182011', 'F182012', 'F182013'],
    value='F101992',
    description='Year',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True)

interact(lights, years = slider);

interactive(children=(SelectionSlider(continuous_update=False, description='Year', options=('F101992', 'F10199…